In [3]:
import pandas as pd
import tensorflow as tf
from pathlib import Path
from defi_lend_eval.modelling import contract

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
# p = Path('/mnt/d/projects/Defi-Lending-Evaluation/data/')
p = Path('/run/media/da/Data/Projects/DeFi-Lending-Evaluation/data/')
fnames = contract.find_data_file(p, '_matrix.csv')
df = contract.read_data(fnames)
df = df.dropna()
display(df.shape)
df.head()

(19392, 17)

,commit,la,ld,ns,nd,nf,ent,nuc,ndev,inter,exp,rexp,sexp,pod,fix,buggy,plat
0,ca96d695,966,37,3,7,14,1.332095,62,1,64.008765,210,0.029312,188,1.000000,False,False,88mph_matrix
1,1d02a6bb,7,7,3,4,5,1.220036,29,1,294.411611,117,0.004706,44,1.000000,False,False,88mph_matrix
2,33639778,24751,115,2,3,44,1.089848,39,2,569.835787,91,0.020129,26,0.487179,False,True,88mph_matrix
3,2383726a,3,1,1,1,1,0.026486,59,1,118.513333,152,0.402389,104,1.000000,True,False,88mph_matrix
4,d01b89b9,31,19,3,3,11,3.046779,10,1,16.237778,77,0.006936,26,1.000000,True,False,88mph_matrix


In [5]:
fnames = contract.find_data_file(p, 'commits.json')
df1 = contract.read_data(fnames, 'json')
df1 = df1.dropna()
df1.head()

,commit,msg,changes,buggy,plat
0,f57cf8a6,Add storage gaps to Rescuable and Sponsorable,diff --git a/contracts/libs/Rescuable.sol b/co...,False,88mph_buggy_commits
1,0a0c5a83,Fixed best practices 10,diff --git a/contracts/DInterest.sol b/contrac...,True,88mph_buggy_commits
2,b5b618bc,Update deploy scripts for money markets & Dump...,diff --git a/contracts/mocks/VaultMock.sol b/c...,False,88mph_buggy_commits
3,005cb00a,Refactor test dependencies,diff --git a/test/DInterest.test.js b/test/DIn...,False,88mph_buggy_commits
4,2fdfecf9,Fix ERC20Wrapper::_transfer(),diff --git a/contracts/libs/ERC20Wrapper.sol b...,False,88mph_buggy_commits


In [6]:
# display(df1.shape)
df1 = df1.drop(['plat'], axis=1)
df1.head()


,commit,msg,changes,buggy
0,f57cf8a6,Add storage gaps to Rescuable and Sponsorable,diff --git a/contracts/libs/Rescuable.sol b/co...,False
1,0a0c5a83,Fixed best practices 10,diff --git a/contracts/DInterest.sol b/contrac...,True
2,b5b618bc,Update deploy scripts for money markets & Dump...,diff --git a/contracts/mocks/VaultMock.sol b/c...,False
3,005cb00a,Refactor test dependencies,diff --git a/test/DInterest.test.js b/test/DIn...,False
4,2fdfecf9,Fix ERC20Wrapper::_transfer(),diff --git a/contracts/libs/ERC20Wrapper.sol b...,False


In [7]:
used_df = df1[['msg', 'buggy']]
used_df.head()

,msg,buggy
0,Add storage gaps to Rescuable and Sponsorable,False
1,Fixed best practices 10,True
2,Update deploy scripts for money markets & Dump...,False
3,Refactor test dependencies,False
4,Fix ERC20Wrapper::_transfer(),False


In [8]:
import re
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

def lower_case(txt):
    """ turn all letters into lower case """
    return str(txt).lower()

def rm_special_words(txt):
    """ remove all sepcial words """
    # remove the urls
    txt = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                 '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', txt)
    # remove the @ and references
    txt = re.sub(r"(@[A-Za-z0-9_]+)","", txt)
    # remove special charaecters
    txt = re.sub(r'[\#\:\)\n\/\_\-\&]','', txt)
    return txt

def filter_words(words):
    """ remove punctuations and stopwords """
    stops = stopwords.words("english")
    words = [w for w in words if (w.isalpha() and w not in stops)]
    return words
    
def stem_words(words):
    """ perform words stemming """
    snowball = SnowballStemmer("english")
    words = [snowball.stem(w) for w in words]
    return words
    
def preprocess(txt):
    """ required preprocessing """
    txt = lower_case(txt)
    txt = rm_special_words(txt)
    words = word_tokenize(txt)
    words = filter_words(words)
    words = stem_words(words)
    return words

In [9]:
used_df['words'] = used_df['msg'].apply(lambda x : preprocess(x))
used_df.head()

<ipython-input-9-bf4f84bf5edb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_df['words'] = used_df['msg'].apply(lambda x : preprocess(x))


,msg,buggy,words
0,Add storage gaps to Rescuable and Sponsorable,False,"[add, storag, gap, rescuabl, sponsor]"
1,Fixed best practices 10,True,"[fix, best, practic]"
2,Update deploy scripts for money markets & Dump...,False,"[updat, deploy, script, money, market, dumper,..."
3,Refactor test dependencies,False,"[refactor, test, depend]"
4,Fix ERC20Wrapper::_transfer(),False,[fix]


In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(used_df, test_size=0.3, random_state=28)

In [11]:
###########################################################################
# 1. Train the Word2Vec model
###########################################################################
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# EMB_FILE_NAME = 'embedding_word2vec.txt'
EMB_FILE_NAME = 'gensim.glove.6B.100d.txt'
# glove2word2vec('glove.6B.100d.txt', EMB_FILE_NAME)

# train the word2vec model
# model = Word2Vec(train['words'].tolist())
# print('Size of the vocabulary:', len(model.wv))

# save word vectors
# model.wv.save_word2vec_format(EMB_FILE_NAME, binary=False)

# load saved embeddings
kv = KeyedVectors.load_word2vec_format(EMB_FILE_NAME, binary=False)
word_vecs = {}
# for key in kv.key_to_index.keys():
for key in kv.vocab.keys():
    word_vecs[key] = kv[key]
print('Display the embedding dictionary as a table:')
# display(pd.DataFrame.from_dict(word_vecs, orient='index'))
    

###########################################################################
# 2. Create the embedding matrix for the Keras model
###########################################################################
import numpy as np
from keras.preprocessing.text import Tokenizer

tknizer = Tokenizer()
tknizer.fit_on_texts(train['words'].tolist())
word_idx = tknizer.word_index
emb_mat = np.zeros((len(word_idx)+1, 100))

for key, val in word_idx.items():
    try:
        emb_mat[val] = word_vecs[key]
    except:
        continue
        
print('\nShape of the embedding matrix:', emb_mat.shape)
       

Display the embedding dictionary as a table:

Shape of the embedding matrix: (7236, 100)


In [12]:
###########################################################################
# 3. Generate training and test data matrix (X_train,X_test) and 
#    corresponding responses (y_train,y_test)
###########################################################################
from tensorflow.keras.preprocessing.sequence import pad_sequences

# process the training inputs
encoded_train_words = tknizer.texts_to_sequences(train['words'].tolist())
max_len = max([len(x) for x in encoded_train_words])
X_train = pad_sequences(encoded_train_words, maxlen=max_len, padding='post')

# process the training outputs
# y_train = pd.get_dummies(train['buggy'])
y_train = train['buggy'].astype(int).to_numpy()

# same processes are applied to test data
encoded_test_words = tknizer.texts_to_sequences(test['words'].tolist())
X_test = pad_sequences(encoded_test_words, maxlen=max_len, padding='post')
# y_test = pd.get_dummies(test['buggy'])
y_test = test['buggy'].astype(int).to_numpy()

print('\nShape of X_train', X_train.shape)
print('Shape of y_train', y_train.shape)
print('Shape of X_test', X_test.shape)
print('Shape of y_test', y_test.shape)
y_test


Shape of X_train (13574, 524)
Shape of y_train (13574,)
Shape of X_test (5818, 524)
Shape of y_test (5818,)


array([0, 0, 0, ..., 0, 0, 0])

In [13]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights

array([0.53390497, 7.87354988])

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from sklearn.metrics import f1_score

word2vec_embedding_layer = Embedding(input_dim=emb_mat.shape[0],
                                     output_dim=emb_mat.shape[1],
                                     weights=[emb_mat],
                                     input_length=max_len,
                                     trainable=False)

model_w2v = Sequential()
model_w2v.add(word2vec_embedding_layer)
model_w2v.add(GRU(32, dropout=0.1, return_sequences=True))
model_w2v.add(GRU(16, dropout=0.1))
model_w2v.add(Dense(1, activation='sigmoid'))

display(model_w2v.summary())

model_w2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_w2v.fit(X_train, y_train, batch_size=64, epochs=20, 
    class_weight={0:0.53390497,1:7.87354988})

model_w2v.save('models/msg_pred_model')
_, acc = model_w2v.evaluate(X_test, y_test)
print("The accuracy is:", acc)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 524, 100)          723600    
_________________________________________________________________
gru (GRU)                    (None, 524, 32)           12864     
_________________________________________________________________
gru_1 (GRU)                  (None, 16)                2400      
_________________________________________________________________
dense (Dense)                (None, 1)                 17        
Total params: 738,881
Trainable params: 15,281
Non-trainable params: 723,600
_________________________________________________________________


None

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/20
213/213 [==============================] - 8s 23ms/step - loss: 0.6936 - accuracy: 0.4868
Epoch 2/20
213/213 [==============================] - 5s 22ms/step - loss: 0.6933 - accuracy: 0.6113
Epoch 3/20
213/213 [==============================] - 5s 22ms/step - loss: 0.6935 - accuracy: 0.4312
Epoch 4/20
213/213 [==============================] - 5s 23ms/step - loss: 0.6934 - accuracy: 0.3963
Epoch 5/20
213/213 [==============================] - 5s 23ms/step - loss: 0.6935 - accuracy: 0.7671
Epoch 6/20
213/213 [==============================] - 5s 23ms/step - loss: 0.6934 - accuracy: 0.0923
Epoch 7/20
213/213 [==============================] - 5s 22ms/step - loss: 0.6933 - accuracy: 0.2406
Epoch 8/20
213/213 [==============================] - 5s 22ms/step - loss: 0.6934 - accuracy: 0.0636
Epoch 9/20
213/213 [==============================] - 

INFO:tensorflow:Assets written to: models/msg_pred_model/assets


INFO:tensorflow:Assets written to: models/msg_pred_model/assets


182/182 [==============================] - 2s 10ms/step - loss: 0.6879 - accuracy: 0.9369
The accuracy is: 0.9369199275970459


In [17]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, f1_score

pred = model_w2v.predict(X_test)
pred = pred >= 0.5
pred = pred.flatten()
display(pred)

y_true = y_test > 0


report = classification_report(y_true, pred)
unique_label = np.unique([y_true, pred])
cmtx = pd.DataFrame(
    confusion_matrix(y_true, pred, labels=unique_label), 
    index=['true:{:}'.format(x) for x in unique_label], 
    columns=['pred:{:}'.format(x) for x in unique_label]
)
print(report)
print(cmtx)

array([False, False, False, ..., False, False, False])

              precision    recall  f1-score   support

       False       0.94      1.00      0.97      5451
        True       0.00      0.00      0.00       367

    accuracy                           0.94      5818
   macro avg       0.47      0.50      0.48      5818
weighted avg       0.88      0.94      0.91      5818

            pred:False  pred:True
true:False        5451          0
true:True          367          0


/home/da/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/da/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/da/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
